In [1]:
import pandas as pd
import numpy as np

# Procesamiento columna 'Descripción'

In [2]:
excel_file = pd.ExcelFile('../data/raw/data.xlsx')
sheets = excel_file.sheet_names
sheets

['Lista de equipos',
 'Historico calif 2_1(10 años)',
 'T-PAP(C)_T-ACE',
 'ARC(I)_ARC(D)',
 'DP',
 'OIL.TAP',
 'DIEL.PF_ DIEL',
 'OIL.CORR',
 'PRO-M',
 'M.OLTC']

In [3]:
import unidecode

def norm_descrpcion(s):
    s = s.lower()
    s = unidecode.unidecode(s)
    return s

converter_descripcion = {}
converter_descripcion['Descripción'] = norm_descrpcion

df_collection = {}
for name in sheets:
    df_collection[name] = pd.read_excel('../data/raw/data.xlsx', sheet_name=name, converters=converter_descripcion)

1. Separar cada fila en cada una de las descripciones. i.e : "realizar analisis dga, humedad y furanos" deben quedar dos filas exactamente iguales pero con cada uno de los registros. -> pd.explode(), explode a esa accion de expandir un dataset con base a una regla.

2. Aplicar un algoritmo de distancia de levehnstein. 

3. Análisis descriptivo que relacione cada equipo con las acciones. i.e: frecuencias de cada accion para cada equipo.

## Separación manual

In [4]:
def separacion_manual(dframe, originales, reemplazos, adiciones):
    df = dframe.copy()
    for i in range(len(originales)):
        index = df['Descripción'] == originales[i]
        if sum(index) > 0:
            instancias = df.loc[index].copy()
            df.loc[index,'Descripción'] = reemplazos[i]
            df = df.append(instancias, ignore_index = True)
            df.loc[len(df) - sum(index):,'Descripción'] = adiciones[i]
    return df




df_collection_separados = {}

excel_file_reemplazos = pd.ExcelFile('../data/reemplazos_manual/reemplazos_manual.xlsx')
sheets_reemplazos = excel_file_reemplazos.sheet_names
df_collection_manuales = {}

converter_manuales = {}
converter_manuales['Originales'] = norm_descrpcion
converter_manuales['Reemplazos'] = norm_descrpcion
converter_manuales['Adiciones'] = norm_descrpcion

for name in sheets_reemplazos:#sheets_reemplazos:
    df_collection_manuales[name] = pd.read_excel('../data/reemplazos_manual/reemplazos_manual.xlsx', sheet_name = name, converters = converter_manuales)
    index_manuales = ~df_collection_manuales[name].loc[:,'Reemplazos'].isna()
    originales = df_collection_manuales[name].loc[index_manuales, 'Originales'].values
    reemplazos = df_collection_manuales[name].loc[index_manuales, 'Reemplazos'].values
    adiciones = df_collection_manuales[name].loc[index_manuales, 'Adiciones'].values
    df_collection_separados[name] = separacion_manual(df_collection[name], originales, reemplazos, adiciones)

## Normalización por palabras claves

In [5]:
palabras_clave = {}
palabras_clave['revisar/cambiar motoventilador'] = ['motoventilador','ventilador','motoventiladores','ventiladores']
palabras_clave['realizar analisis dga y humedad seguimiento'] = ['dga']
palabras_clave['hacer seguimiento generacion anormal de gases'] = ['generacion','energizacion','gases']
palabras_clave['cambiar silica oltc'] = ['silica']
palabras_clave['cambiar filtro oltc'] = ['filtro']
palabras_clave['realizar secado oltc'] = ['secado']
palabras_clave['realizar muestreo-analisis fsco-qco oltc'] = ['aceite','muestra','fsco','qco']
palabras_clave['realizar regeneramiento aceite aislante'] = ['regeneramiento','regeneracion']
palabras_clave['adicionar irgamet39 a 100ppm'] = ['irgamet','pasivador','pasivar']
palabras_clave['temperatura devenado'] = ['calibrar indicador temperatura devanados']
palabras_clave['efectuar prueba de descargas parciales'] = ['descargas parc']
palabras_clave['seguimiento durante montaje'] = ['durante montaje']
palabras_clave['verificar condicion antes energizar oltc'] = ['energizar']
palabras_clave['realizar prueba estimacion humedad whrt'] = ['estimacion humedad']
palabras_clave['cambiar valvula alivio presion'] = ['alivio presion', 'alivio de presion','valvula alivio','valvula de alivio','valvula de sobre presion','valvula sobre presion','valvula de sobrepresion','valvula sobrepresion']
palabras_clave['presion subita'] = ['cambiar rele presion subita']
palabras_clave['revisar/cambiar contador cambiador tapsbiador'] = ['contador']
df_collection_pc = df_collection_separados.copy()
unicos_pc = {}
index_unicos_pc = {}

for name in sheets_reemplazos:
    unicos = pd.unique(df_collection_separados[name]['Descripción'])
    index_unicos = np.zeros(len(unicos))
    for i in range(len(unicos)):
        index_unicos[i] = i
        for key,value in palabras_clave.items():
            for word in value:
                if word in unicos[i]:
                    df_collection_pc[name] = df_collection_pc[name].replace(unicos[i],key)
                    unicos[i] = key
                    break
    unicos_pc[name] = unicos
    index_unicos_pc[name] = index_unicos

## Aplicando el algoritmo de distancia de Levenshtein

### Algoritmo de Levenshtein

In [6]:
def dist_lev(s1,s2):
    dist_matrix = np.zeros((len(s1) + 1, len(s2) + 1))
    dist_matrix[1:,0] = range(1,len(s1) + 1)
    dist_matrix[0,1:] = range(1,len(s2) + 1)
    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            if s1[i - 1] == s2[j - 1]:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1], 
                                    dist_matrix[i, j - 1] + 1)
            else:
                dist_matrix[i,j] = min(dist_matrix[i - 1, j] + 1, 
                                    dist_matrix[i - 1, j - 1] + 1, 
                                    dist_matrix[i, j - 1] + 1)
    return dist_matrix[-1,-1]

### Aplicando y ordenando según distancia

In [7]:
# Extrayendo valores únicos:
ordenados_collection = {}
dist_ordenados_collection = {}
index_ordenados_collection = {}
for name in sheets_reemplazos:
    unicos = pd.unique(unicos_pc[name])
    #Construyendo matriz de distancia de Levenshtein
    dist_unicos = np.zeros((len(unicos), len(unicos)))
    for i in range(len(unicos)):
        for j in range(i + 1, len(unicos)):
            dist_unicos[i, j] = dist_lev(unicos[i], unicos[j])
            dist_unicos[j,i] = dist_unicos[i,j] 
    #Inizializar varibales
    # dist_unicos = np.array([[0,8,8,10.5],[1,0,4,3],[2,5,0,6],[8,10,7,0]])
    # unicos = [1,2,3,4]
    ordenados = ['']*len(unicos)
    dist_ordenados = np.zeros(len(unicos))
    index_ordenados = np.zeros(len(unicos))
    #Ubicando primer elemento
    max_dist_index = np.argmax(dist_unicos)
    max_dist_index = np.unravel_index(max_dist_index, np.shape(dist_unicos))
    row = max_dist_index[0]
    ordenados[0] = unicos[row]
    index_ordenados[0] = row
    dist_ordenados[0] = 0
    dist_unicos[row,:] = np.inf
    #Diagonal igual a infinito
    for i in range(len(unicos)):
        dist_unicos[i,i] = np.inf
    #Ubicando el resto de los elementos
    for i in range(len(unicos) - 1):
        min_dist = np.min(dist_unicos[:,row])
        dist_ordenados[i + 1] = dist_ordenados[i] + min_dist
        row = np.argmin(dist_unicos[:,row])
        ordenados[i + 1] = unicos[row]
        index_ordenados[i + 1] = row
        dist_unicos[row,:] = np.inf
    #Ubicándolos por hoja
    ordenados_collection[name] = ordenados
    dist_ordenados_collection[name] = dist_ordenados
    index_ordenados_collection[name] = index_ordenados


### Escribiendo los resultados en Excel

In [8]:
import xlsxwriter
workbook = xlsxwriter.Workbook('../data/reemplazos_manual/unicos_ordenados.xlsx')
for name in sheets_reemplazos:
    worksheet = workbook.add_worksheet(name)
    for i in range(len(ordenados_collection[name])): 
        worksheet.write(i,0,ordenados_collection[name][i])
        worksheet.write(i,1,ordenados_collection[name][i])
        worksheet.write(i,2,dist_ordenados_collection[name][i])
        worksheet.write(i,3,index_ordenados_collection[name][i])

workbook.close()

# Separación por equipos

In [25]:
equipos_collection = {}
equipos_unicos = pd.unique(df_collection['Lista de equipos']['Equipo'])

for equipo in equipos_unicos:
   df_equipo = pd.DataFrame()
   for sheet in sheets_reemplazos:
      for i in range(len(df_collection_pc[sheet])):
         instancia = df_collection_pc[sheet].iloc[i].copy()
         if instancia['Equipo'] == equipo:
            df_equipo = df_equipo.append(instancia)
   equipos_collection[equipo] = df_equipo



In [27]:

equipos_collection[810451]

,Autor del aviso,Aviso,Clase de aviso,Creado el,Denominación,Descripción,Equipo,Fecha de cierre,Fin deseado,Indicador ABC,Inicio deseado,Perfil catálogo,Prioridad,Pto.tbjo.resp.,Sociedad CO,Status sistema,Status usuario,Ubicac.técnica,Área de empresa
2,FAHERRENO,233587.0,N4,2011-04-13,Tebsa 3 60 MVA 220/110/46 kV Fase T,realizar analisis dga y humedad seguimiento,810451.0,2011-10-11,2011-09-30,NaN,2011-09-30,N-AUTOTR,3.0,38B40005,TRAN,MECE ORAS,INIC COND,TBS_220ATR03-T,TBS
42,FAHERRENO,322430.0,N4,2013-05-21,Tebsa 3 60 MVA 220/110/46 kV Fase T,revisar/cambiar motoventilador,810451.0,2014-09-09,2014-08-24,NaN,2014-08-24,N-AUTOTR,3.0,38C10001,TRAN,MECE ORAS,CPRO ENPT,TBS_220ATR03-T,TBS
43,FAHERRENO,322431.0,N4,2013-05-21,Tebsa 3 60 MVA 220/110/46 kV Fase T,realizar muestreo y analisis furanos,810451.0,2013-11-26,2013-11-06,NaN,2013-11-06,N-AUTOTR,2.0,38C10001,NaN,MECE ORAS,INIC COND,TBS_220ATR03-T,TBS
47,AFERNANDEZ,337262.0,N4,2013-10-04,Tebsa 3 60 MVA 220/110/46 kV Fase T,revisar/cambiar motoventilador,810451.0,2014-09-09,2014-08-24,NaN,2014-08-24,N-AUTOTR,3.0,38C10001,TRAN,MECE ORAS,CPRO COND ENPT,TBS_220ATR03-T,TBS
116,HARGUELLO,321252.0,N4,2013-05-06,Tebsa 3 60 MVA 220/110/46 kV Fase T,revisar senalizacion indicador cambiador,810451.0,2013-07-11,2013-06-11,NaN,2013-06-02,N-AUTOTR,3.0,38C10001,NaN,MECE ORAS,INIC COND,TBS_220ATR03-T,TBS
118,FAHERRENO,322426.0,N4,2013-05-21,Tebsa 3 60 MVA 220/110/46 kV Fase T,realizar calibracion termometros,810451.0,2014-09-10,2014-08-24,NaN,2014-08-24,N-AUTOTR,3.0,38C10001,TRAN,MECE ORAS,CPRO COND ENPT,TBS_220ATR03-T,TBS
144,AFERNANDEZ,337348.0,N4,2013-10-07,Tebsa 3 60 MVA 220/110/46 kV Fase T,realizar muestreo-analisis fsco-qco oltc,810451.0,2014-11-12,2014-10-18,NaN,2014-10-18,N-AUTOTR,4.0,38C10001,TRAN,MECE ORAS,INIC COND ENPT,TBS_220ATR03-T,TBS
197,AFERNANDEZ,377391.0,N4,2014-08-28,Tebsa 3 60 MVA 220/110/46 kV Fase T,hacer tratamiento metalmecanico-pintura,810451.0,2015-09-03,2015-08-23,NaN,2015-08-23,S-TRAFOS,4.0,38C10001,TRAN,MECE ORAS,INIC ENPT,TBS_220ATR03-T,TBS
198,AFERNANDEZ,377400.0,N4,2014-08-28,Tebsa 3 60 MVA 220/110/46 kV Fase T,cambiar termometro temperatura dev. at,810451.0,2014-11-12,2014-10-18,NaN,2014-10-18,N-AUTOTR,3.0,38C10001,TRAN,MECE ORAS,INIC ENPT,TBS_220ATR03-T,TBS
359,AFERNANDEZ,5408509.0,N4,2018-05-09,Tebsa 3 60 MVA 220/110/46 kV Fase T,realizar tratamiento metalmecanico,810451.0,2018-07-06,2018-06-15,A,2018-06-14,S-TRATAM,4.0,38C10001,TRAN,MECE ORAS,INIC ENPT,TBS_220ATR03-T,TBS
